In [1]:
import pandas as pd
import numpy as np
import selenium
from selenium import webdriver
import time
import requests
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import warnings
warnings.filterwarnings('ignore')

In [2]:
response=requests.get('https://www.happyeasygo.com')
response

<Response [200]>

In [3]:
#loading driver
driver=webdriver.Chrome(r"chromedriver.exe")
# loading website in the chrome driver
driver.get("https://www.happyeasygo.com/flights/DEL-BOM/2023-01-20?tripType=0&adults=1&childs=0&baby=0&cabinClass=Economy&airline=&carrier=")


In [4]:
Source=['DEL','BLR','HYD']
Destination=['BOM','LKO','BOM']
Date=['2023-01-20','2023-01-21','2023-01-22','2023-01-23','2023-01-24','2023-01-25','2023-01-26','2023-01-27','2023-01-28','2023-01-29','2023-01-30','2023-01-31','2023-02-01','2023-02-02','2023-02-03']

In [6]:
airline=[]
doj=[]
origin=[]
f_Destination=[]
dep_time=[]
arr_time=[]
duration=[]
total_stops=[]
price=[]

capa = DesiredCapabilities.CHROME
capa["pageLoadStrategy"] = "eager" #since webpage has long loading time. This command will start scraping once 
driver = webdriver.Chrome(desired_capabilities=capa)  #the html is loaded.

for i in Date:
    for j in Source:
        for k in Destination:
            if j!=k:
                url='https://www.happyeasygo.com/flights/{}-{}/{}?tripType=0&adults=1&childs=0&baby=0&cabinClass=Economy&airline=&carrier='.format(j,k,i)
                driver.get(url)
                time.sleep(5)
                
               #first we will scrape the data for non stop flights
                driver.find_element(By.XPATH,"//div[@class='lowest-price']").click()
                
                for l in driver.find_elements(By.XPATH,"//div[@class='flight-title P-ellipsis']"):
                    airline.append(l.text)
                    doj.append(i)
                    origin.append(j)
                    f_Destination.append(k)
                    
                for l in driver.find_elements(By.XPATH,"//div[@class='o-vendor flight-stop ']"):
                    total_stops.append(l.text)
            
                for l in driver.find_elements(By.XPATH,"//span[@class='o-time-bold o-time_line']"):
                    dep_time.append(l.text)
                
                for l in driver.find_elements(By.XPATH,"//span[@class='o-time-bold']"):
                    arr_time.append(l.text.replace('\n+1',''))
                
                for l in driver.find_elements(By.XPATH,"//div[@class='o-vendor fly-time']"):
                    duration.append(l.text)
                    
                for l in driver.find_elements(By.XPATH,"//span[@class='price-from-second']"):
                    price.append(l.text.replace('₹ ','').replace(',',''))

In [7]:
print(len(airline),len(doj),len(origin),len(f_Destination),len(total_stops),len(dep_time),len(arr_time),len(duration),len(price))

2968 2968 2968 2968 2943 2968 2968 2968 2968


In [10]:
df=pd.DataFrame({'Airline':airline[:2900],'Date of Journey':doj[:2900],'Source':origin[:2900],'Destination':f_Destination[:2900],'Departure Time':dep_time[:2900],'Arrival Time':arr_time[:2900],'Duration':duration[:2900],'Total Stops':total_stops[:2900],'Price':price[:2900]})
df

,Airline,Date of Journey,Source,Destination,Departure Time,Arrival Time,Duration,Total Stops,Price
0,G8330,2023-01-20,DEL,BOM,21:00,23:15,2h15m,non-stop,3800
1,G82501,2023-01-20,DEL,BOM,02:40,04:50,2h10m,non-stop,4403
2,G8354,2023-01-20,DEL,BOM,22:30,00:40,2h10m,non-stop,4403
3,G8530,2023-01-20,DEL,BOM,06:00,08:10,2h10m,non-stop,4403
4,UK975,2023-01-20,DEL,BOM,05:45,08:00,2h15m,non-stop,4535
...,...,...,...,...,...,...,...,...,...
2895,UK850,2023-02-03,BLR,BOM,19:55,21:40,1h45m,non-stop,5616
2896,I51532,2023-02-03,BLR,LKO,18:45,21:10,2h25m,non-stop,3649
2897,G8808,2023-02-03,BLR,LKO,16:05,18:25,2h20m,non-stop,3915
2898,I52472,2023-02-03,BLR,LKO,06:50,09:15,2h25m,non-stop,4440


In [11]:
#Saving Dataframe into CSV Format
df.to_csv('Flightsdata.csv')